In [1]:
from functions import *
from ipywidgets import HBox, HTML, Dropdown, Label, Layout,Image
import tkinter as tk
from IPython.display import display, Markdown, clear_output, IFrame,Javascript
from pyvis.network import Network
pathways_name=pd.read_csv("data/pathways.tsv", sep='\t')["pathway_name"]
last_selection=pathways_name
display(HTML("<style>.container { width:100% !important; }</style>"))


HTML(value='<style>.container { width:100% !important; }</style>')

In [2]:
display(HTML("<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>"))
text=Label(value="")
text.add_class("left-spacing-class")
menu = Dropdown(
       options=list(pathways_name.values),
       value=list(pathways_name.values)[25],
       description='Pathway:')
sidebyside=HBox([menu, HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells([2])"><input type="submit" id="toggleButton" value="Run"></form>'''),text])

display(sidebyside)

HTML(value='<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>')

In [5]:
skip_calcs=False
selection=menu.value
pathway_edges=read_pathway(selection)
if (len(pathway_edges)==0):
    text.value="Nodes not found in eset file"
    selection=last_selection
    skip_calcs=True
else:
    text.value="Please wait..."
    skip_calcs=False
    last_selection=selection
pathway_edges=read_pathway(selection)
if skip_calcs==False:
    adj_matrix,nodes_renamed,inv_nodes_renamed=build_adj(pathway_edges)
    G = nx.from_numpy_matrix(adj_matrix)
    triad_cliques=get_triad(G)
    weighted_edges=calculate_weighted_edges(triad_cliques, adj_matrix,inv_nodes_renamed)
    text.value=""
    to_remove=[]
    for x in weighted_edges.items():
        zeros=0
        ones=0
        minus=0
        for z in x[1]:
            if (z[1]==0):
                zeros+=1
            elif (z[1]==1):
                ones+=1
            else:
                minus+=1
        if (ones==0):
            if (minus==0):
                to_remove.append(x[0])
            else:
                if (zeros/minus>1):
                    to_remove.append(x[0])

    relabel={}
    for e,node in enumerate( G.nodes()):
        relabel[e]=str(inv_nodes_renamed[node])
net=Network(height="700px",notebook=True,directed=True,width="100%", bgcolor='#222222', font_color='white')
for i,node in relabel.items():
    net.add_node(str(node))

for edge in pathway_edges.values:
        if(edge[2]==-1):
            net.add_edge(str(edge[0]), str(edge[1]), color="yellow")
        else:
            net.add_edge(str(edge[0]), str(edge[1]))
for triad in triad_cliques:
    for i,x in enumerate(triad):
        for j,y in enumerate(triad):
            if ((str(inv_nodes_renamed[triad[i]])+","+str(inv_nodes_renamed[triad[j]])) in to_remove) or ((str(inv_nodes_renamed[triad[j]])+","+str(inv_nodes_renamed[triad[i]])) in to_remove):
                color="red"
                size=10
            else:
                color="green"
                size=3
            weight=pathway_edges[(pathway_edges[0]==inv_nodes_renamed[triad[i]]) & (pathway_edges[1]==inv_nodes_renamed[triad[j]])]
            if (weight.empty):
                continue
            weight=int(weight[2].values)
            if (weight==1):
                net.add_edge(str(inv_nodes_renamed[triad[i]]), str(inv_nodes_renamed[triad[j]]), color=color, width=size,title="Express")
            else:
                net.add_edge(str(inv_nodes_renamed[triad[i]]), str(inv_nodes_renamed[triad[j]]), color=color, width=size,title="Suppress")
           # elif(adj_matrix[x][y]==-1):
            #    net.add_edge(str(inv_nodes_renamed[triad[j]]), str(inv_nodes_renamed[triad[i]]), color=color, width=size)
#net.show_buttons(filter_=["physics"])
net.hrepulsion(node_distance=120, central_gravity=0.0, spring_length=100, spring_strength=0, damping=0.09)
net.show("graph.html")


